In [19]:
import json
import spacy
import numpy as np
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity
from spacy.training.example import Example


In [63]:
data_txt  = [
    ("I love this brand of toothpaste", {"entities": [(21, 31, "PRODUCT")]}),
    ("The coffee maker makes great coffee", {"entities": [(4, 15, "PRODUCT")]}),
    ("I need to buy a new printer", {"entities": [(20, 27, "PRODUCT")]}),
    ("This lotion is very moisturizing", {"entities": [(5, 11, "PRODUCT")]}),
    ("I use this brand of shampoo all the time", {"entities": [(20, 27, "PRODUCT")]}),
    ("This camera takes high-quality photos", {"entities": [(5, 11, "PRODUCT")]}),
    ("I want to buy a new pair of headphones", {"entities": [(28, 38, "PRODUCT")]}),
    ("This brand of cereal is my favorite", {"entities": [(14, 20, "PRODUCT")]}),
    ("I need to get more printer paper", {"entities": [(27, 32, "PRODUCT")]}),
    ("This brand of juice is really good", {"entities": [(14, 19, "PRODUCT")]}),
    ("These headphones have great sound quality", {"entities": [(6, 16, "PRODUCT")]}),
    ("I want to buy a new laptop", {"entities": [(20, 26, "PRODUCT")]}),
    ("This soap smells really nice", {"entities": [(5, 9, "PRODUCT")]}),
    ("I love this brand of tea", {"entities": [(21, 24, "PRODUCT")]}),
    ("This brand of chips is really tasty", {"entities": [(14, 19, "PRODUCT")]}),
    ("I need more printer ink", {"entities": [(20, 23, "PRODUCT")]}),
    ("This jacket is really warm", {"entities": [(5, 11, "PRODUCT")]}),
    ("I want to buy a new phone", {"entities": [(20, 25, "PRODUCT")]}),
    ("This brand of cookies is delicious", {"entities": [(14, 21, "PRODUCT")]}),
    ("This brand of paper towels is my favorite", {"entities": [(14, 26, "PRODUCT")]}),
    ("I need to buy a new pair of shoes", {"entities": [(28, 33, "PRODUCT")]}),
    ("This brand of yogurt is really good", {"entities": [(14, 20, "PRODUCT")]}),
    ("These headphones are very comfortable", {"entities": [(6, 16, "PRODUCT")]}),
    ("I love this brand of chips", {"entities": [(21, 27, "PRODUCT")]}),
    ("This brand of bread is my favorite", {"entities": [(14, 19, "PRODUCT")]}),
    ("This camera has great features", {"entities": [(5, 11, "PRODUCT")]}),
    ("I want to try this new brand of coffee", {"entities": [(32, 38, "PRODUCT")]}),
    ("This brand of ice cream is really tasty", {"entities": [(14, 23, "PRODUCT")]}),
    ("I need to get more laundry detergent", {"entities": [(27, 36, "PRODUCT")]}),
    ("This brand of cheese is really good", {"entities": [(14, 20, "PRODUCT")]}),
    ("I want to buy a new pair of running shoes", {"entities": [(28, 41, "PRODUCT")]}),
]


In [50]:
 def load_mappings(dataset):
    f = open(dataset, 'r',encoding='utf-8')
    model= dict({})
    for line in f:
        tem=line.split()
        vtem=[]
        for i in range(1,len(tem)):
            vtem.append( float(tem[i].strip()))
        vec1=np.array(vtem)
        model.update({tem[0].strip():vec1})
    return model


In [51]:
def train (label, train_data, model):
    # Add the new label to the entity recognizer
    # Load the pre-trained model
    nlp = spacy.load(model)

    nlp_train = spacy.load(model)

    ner = nlp_train.get_pipe("ner")
    ner.add_label(label)

    # Fine-tune the model
    optimizer = nlp_train.resume_training()
    for i in range(10):
        for example in train_data:
            doc = nlp_train(example[0])
            gold = Example.from_dict(doc, example[1])
            nlp_train.update([gold], sgd=optimizer)
    return nlp_train

In [52]:
def get_product(text : str, train_model) -> str :
    # Test the model train
    list_test = []
#   label = ["PRODUCT","PRICE"]
    doc = train_model(text)        
    for ent in doc.ents:
        #if ent.label_ in label:
        list_test.append([ent.text, ent.label_])
    return list_test

In [54]:
# get_product_category 
def get_product_category(my_product, my_list):
    for i in my_product:
        if i[1] == "PRODUCT":
            product = i[0]
    if product in load_model: 
        embeddings = []
        v1 = load_model[product.strip()]
        embeddings.append(v1)
        for category in my_list:
            embeddings.append(load_model[category.strip()])
        cos_sim = cosine_similarity( [embeddings[0]], embeddings[1:])
        return my_list[np.argmax(cos_sim)]
    return 0

In [55]:
def Workflow(label, model, train_data, test_text):
    nlp_train = train(label, train_data, model)
    nlp_test = get_product(test_text,nlp_train)
    return nlp_test



In [56]:
def catch_cat(predict, data):
    for i in data:
        for u in data:
            if u['category'] == predict:
                return u['item_attributes']
            


In [57]:
def store_att(att,my_product):
    my_dict = {'PRODUCT':'','item_attributes':{}}
    for i in my_product:
        if i[-1] == 'PRODUCT':
            my_dict['PRODUCT'] = i[0]
        if i[-1] in att:
            my_dict['item_attributes'][i[-1]] = i[0]
    return my_dict

In [58]:
#load w2v
load_model = load_mappings("glove.6B.300d.txt")

In [59]:
type(load_model)

dict

In [81]:
if __name__ == '__main__':
    # Define the new entity label
    LABEL = "PRODUCT"
    # Define model
#     model_name = "en_core_web_sm"
    # Define the training data
    model_name = "en_core_web_trf"

    #with open("train_Data.txt") as data_train:
      #  data_txt = data_train.read()
#     TRAIN_DATA = data_txt
    TRAIN_DATA = [
    (
        "This is a great product",
        {
            "entities": [(10, 17, "PRODUCT")]
        },
    ),
    (
        "I love this shampoo",
        {
            "entities": [(12, 19, "PRODUCT")]
        },
    ),
    (
        "The phone has a great camera",
        {
            "entities": [(4, 9, "PRODUCT")]
        },
    ),
    (   "My favorite phone is red",
        {
            "entities": [(21, 24, "COLOR")]
        },
        )
    ]
    category_list = ["phone", "clothes"]
    
    test_text = "My favorite phone is red."
    model = train(LABEL,data_txt,model_name)
    my_product = get_product(test_text, model)
    predict = get_product_category(my_product, category_list)
    print(f"item: {my_product}")
    print(f"category: {predict}")
    # read category-list
    with open('category-list.json') as file:
        category_list = json.load(file)

    catch = catch_cat(predict,category_list)
    stores = store_att(catch,my_product)
    # Write json data 
    with open("cate_list.json", "w") as json_file:  
    # Sử dụng json.dump() để ghi dữ liệu vào file   
        json.dump(stores, json_file) 

/home/minh/anaconda3/lib/python3.10/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/home/minh/anaconda3/lib/python3.10/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "The coffee maker makes great coffee" with entities "[(4, 15, 'PRODUCT')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/home/minh/anaconda3/lib/python3.10/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "I love this brand of chips" with entities "[(21, 27, 'PRODUCT')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('

UnboundLocalError: local variable 'product' referenced before assignment

In [77]:
my_product

[]

In [29]:
my_product

[['1000$', 'MONEY']]

In [51]:
with open("train_Data.txt","r") as f:
    data_js = json.load(f)
print(data_js)

JSONDecodeError: Expecting value: line 2 column 5 (char 6)